<div class='alert alert-info' style='text-align:center'><h1>Applying filters to Chest X-Rays</h1>- yet another chest x-ray processing notebook -</div>

#### While exploring methods to better visualize lung tissue, I found some useful filters from sklearn.filters I thought I'd share.
#### In this notebook we'll grab an image and apply a filter or two. Params can be tweaked to get some pretty unique views.

- In this case, we'll use two modules from *sklearn.exposure* .. **equalize_hist** and **equalize_adapthist**
- The filters are **unsharp_mask**, **meijering**, **sato**, **scharr**, and **hessian** from *sklearn.filters*

In [ ]:
# You will need to run this command if opening a JPG encoded DICOM file
#!conda install gdcm -c conda-forge -y

### Load stuff and define some functions

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
from skimage.filters import unsharp_mask, meijering, sato, scharr, hessian
from skimage.exposure import exposure, equalize_hist, equalize_adapthist
import cv2 as cv

In [ ]:
# This function gets the first image path in a StudyInstanceUID directory in the train set
def get_image_by_study_id(study_id):
    base_path = "/kaggle/input/siim-covid19-detection/"
    study_path = base_path + "train/" + study_id + "/"
    images = []
    for subdir, dirs, files in os.walk(study_path):
        for file in files:     
            image = os.path.join(subdir, file)
            if os.path.isfile(image):
                return image
    return "none"

In [ ]:
# Load a DICOM file and get the pixels
def load_image(study_id):
    img_file = get_image_by_study_id(study_id)
    image = pydicom.dcmread(img_file)
    pixels = image.pixel_array

    min_pixel = np.min(pixels)
    max_pixel = np.max(pixels)

    if image.PhotometricInterpretation == "MONOCHROME1":
        pixels = max_pixel - pixels
    else:
        pixels = pixels

    return pixels

In [ ]:
# Apply filters. Tweak params here.
def apply_filter(f, img):
    if f == 'equalize_hist':
        img = equalize_hist(img, nbins=256, mask=None)
        
    if f == 'equalize_adapthist': 
        img = equalize_adapthist(img, kernel_size=None, clip_limit=0.01, nbins=256)
        
    if f == 'unsharp_mask':
        img = unsharp_mask(img, radius=5, amount=2)
        
    if f == 'meijering':
        img = meijering(img, sigmas=range(1, 10, 2), alpha=None, black_ridges=True, mode='reflect', cval=0)
        
    if f == 'sato':
        img = sato(img, sigmas=range(1, 10, 2), black_ridges=True, mode='reflect', cval=0)
        
    if f == 'scharr':
        img = scharr(img, mask=None, axis=None, mode='reflect', cval=0.0)
        
    if f == 'hessian':
        img = hessian(img, sigmas=range(1, 10, 2), scale_range=None, scale_step=None, alpha=0.5, beta=0.5, gamma=15, black_ridges=True, mode='reflect', cval=0)
    
    if f == 'threshold_isodata':
        img = threshold_isodata(image=img, nbins=256, return_all=False, hist=None)
    
    return img

In [ ]:
# Display the original image and the processed image
def plot_images(title, image, image_processed):
    # Plot both images
    fig, axes = plt.subplots(nrows=1, ncols=2,sharex=True, sharey=True, figsize=(12, 12))
    ax = axes.ravel()
    ax[0].imshow(image, cmap=plt.cm.gray)
    ax[0].set_title('Original image')
    ax[1].imshow(image_processed, cmap=plt.cm.gray)
    ax[1].set_title(title)
    for a in ax:
        a.axis('off')
    fig.tight_layout()
    plt.show()

### Load an image and apply equalization
- We'll use Histogram Equalization and Contrast Limited Adaptive Histogram Equalization (CLAHE)
- Hist EQ gives a nice contrasty view of a chest x-ray.
- CLAHE seems to be more balanced and 'wide' if the input image isn't poorly balanced.

In [ ]:
# Load an image using the Study_ID
image = load_image('013d698aeecb')

In [ ]:
# Apply Histogram Equalization
image_out = apply_filter('equalize_hist', image)
plot_images('Histogram Equalization',image, image_out)

In [ ]:
# Apply Adaptive Histogram Equalization (CLAHE)
image_out = apply_filter('equalize_adapthist', image)
plot_images('CLAHE',image, image_out)

### Put an unsharp mask on the original image
- Why is it called unsharp mask when it actually sharpens the image? (because the first step of unsharp is to blur, which is .. um .. unsharp)
- The unsharpened image is more crisp and defined.

In [ ]:
# Apply unsharp mask
image_out = apply_filter('unsharp_mask', image_out)
plot_images('Unsharp',image, image_out)

### Sklearn has a lot of edge detection filters. Here's a few I found useful.
- Play around with the default params in the apply_filter method to get some interesting results.

In [ ]:
# Apply Meijering and then Hist EQ to brighten it up
image_out = apply_filter('meijering', image)
image_out = apply_filter('equalize_hist', image_out)
plot_images('Meijering',image, image_out)

In [ ]:
# Apply Sato and Hist EQ
image_out = apply_filter('sato', image)
image_out = apply_filter('equalize_hist', image_out)
plot_images('Sato',image, image_out)

In [ ]:
# Scharr is similar to Sobel, maybe a little faster?
image_out = apply_filter('scharr', image)
image_out = apply_filter('equalize_hist', image_out)
plot_images('Scharr',image, image_out)

In [ ]:
# Apply Hessian and CLAHE
image_out = apply_filter('hessian', image)
image_out = apply_filter('equalize_adapthist', image_out)
plot_images('Hessian',image, image_out)

#### As you can see, there *are* methods to demonstrate tissue areas with more/less contrast. Some are noisy. Some are not.

- Stacking filters and using various parameter settings can yield interesting results.

Here are some other processing notebooks I made:
- Lung Segmentation Without CNN -> https://www.kaggle.com/davidbroberts/lung-segmentation-without-cnn
- Rib supression on Chest X-Rays -> https://www.kaggle.com/davidbroberts/rib-suppression-poc
- Manual DICOM VOI LUT -> https://www.kaggle.com/davidbroberts/manual-dicom-voi-lut
- Apply Unsharp Mask to Chest X-Rays -> https://www.kaggle.com/davidbroberts/unsharp-masking-chest-x-rays
- Cropping Chest X-Rays -> https://www.kaggle.com/davidbroberts/cropping-chest-x-rays
- Bounding Boxes on Cropped Images -> https://www.kaggle.com/davidbroberts/bounding-boxes-on-cropped-images
- Visualizing Chest X-Ray bit planes -> https://www.kaggle.com/davidbroberts/visualizing-chest-x-ray-bitplanes
- DICOM full range pixels as CNN input -> https://www.kaggle.com/davidbroberts/dicom-full-range-pixels-as-cnn-input
- Standardizins Chest X-Ray Dataset Exports -> https://www.kaggle.com/davidbroberts/standardizing-cxr-datasets